In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
#from cifar import load_CIFAR10
#plt.rcParams['figure.figsize'] = (10.0, 8.0) 
#
#cifar10_dir = './cifar10/cifar-10-batches-py'
#X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [ ]:
y_train=list(y_train)
y_test=list(y_test)

In [ ]:
X_train=X_train.reshape((X_train.shape[0], 32, 32, 3))#.transpose(0,3,1,2)
X_test=X_test.reshape((X_test.shape[0], 32, 32, 3))#.transpose(0,3,1,2)
print X_train.shape
for i in range(len(y_train)):
    a=np.zeros(10)
    a[y_train[i]]=1.0
    y_train[i]=a
y_train=np.array(y_train)
print y_train.shape
for i in range(len(y_test)):
    a=np.zeros(10)
    a[y_test[i]]=1.0
    y_test[i]=a
y_test=np.array(y_test)

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
from theano import tensor as T
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

In [ ]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

Так задаётся архитектура нейронки

In [ ]:
#предсказание нейронки (theano-преобразование)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### дальше вы могли бы просто
* задать функцию ошибки вручную
* посчитать градиент ошибки по all_weights
* написать updates
* но это долго, а простой шаг по градиенту - не самый лучший смособ оптимизировать веса

Вместо этого, опять используем lasagne

### Вот и всё, пошли её учить

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
y_train=y_train.astype(np.int32)
y_test=y_test.astype(np.int32)
print y_train[0]
print X_train.shape

# Процесс обучения

In [ ]:
import time
from tqdm import tqdm

#num_epochs = 10 #количество проходов по данным

#batch_size = 10 #размер мини-батча

model.fit(X_train,y_train,batch_size=10,nb_epoch=10,verbose=1)
res= model.evaluate(X_test,y_test)
    
    # In each epoch, we do a full pass over the training data:
#    for batch in tqdm(iterate_minibatches(X_train, y_train,batch_size)):
#        inputs, targets = batch
#        
#        model.train_on_batch(inputs.reshape((-1,32,32,3)),targets.reshape((-1,10)))
    # And a full pass over the validation data:
#    val_acc = 0
#    val_batches = 0
#    for batch in iterate_minibatches(X_test, y_test, batch_size):
#        inputs, targets = batch
#        val_acc += model.evaluate(inputs.reshape((-1,32,32,3)), targets.reshape((-1,10)))
#        val_batches += 1

    
    # Then we print the results for this epoch:
#    print("Epoch {} of {} took {:.3f}s".format(
#        epoch + 1, num_epochs, time.time() - start_time))


#    print("  validation accuracy:\t\t{:.2f} %".format(
#        val_acc / val_batches * 100))

In [ ]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 80:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

## Заполните форму

https://goo.gl/forms/FsANPB1jSqmX1JBJ3